In [ ]:
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from itertools import product

In [ ]:
#Helper functions
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

#Cnn Model
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)


    self.fc1=nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2=nn.Linear(in_features=120, out_features=60)
    self.out=nn.Linear(in_features=60, out_features=10)


  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x, kernel_size=2, stride=2)
    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x, kernel_size=2, stride=2)
    x = torch.flatten(x, start_dim=1)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.out(x)

    return x

# import data
train_set = torchvision.datasets.FashionMNIST(root="./data", train=True, download=True,
                                              transform=transforms.ToTensor())

device = ('cuda' if torch.cuda.is_available() else 'cpu')


#Hyperparameters
parameters=dict(
    lr=[0.01,0.001],
    batch_size=[32,64,128],
    shuffle=[True, False]
)
param_values=[v for v in parameters.values()]

100%|██████████| 26421880/26421880 [00:01<00:00, 17026449.50it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272209.48it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5044965.78it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22123234.62it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:

#Training loop
model_t=CNN()
for run_id, (lr, batch_size, shuffle) in enumerate(product(*param_values)):
  print("run_id:", run_id +1)
  model=CNN().to(device)
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
  optimizer=opt.Adam(model.parameters(),lr=lr)
  criterion=torch.nn.CrossEntropyLoss()
  comment=f'batch_size={batch_size} lr={lr} shuffle={shuffle}'
  tb=SummaryWriter(comment=comment)
  images,labels=next(iter(train_loader))
  grid=torchvision.utils.make_grid(images)
  tb.add_image("images", grid)
  tb.add_graph(model_t, images)
  for epoch in range(5):
    total_loss=0
    total_correct=0
    for images, labels in train_loader:
      images, labels=images.to(device),labels.to(device)
      preds=model(images)

      loss=criterion(preds, labels)
      total_loss +=loss.item()
      total_correct+=get_num_correct(preds, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #Add scalar¶ ... Scalar value is the most simple data
      #type to deal with. Mostly we save the loss value of each training step, or the accuracy after each epoch.
    tb.add_scalar("loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)

    print("batch_size:", batch_size, "lr:", lr,"shuffle:",shuffle)
    print("epoch:",epoch, "total_corrct:",total_correct,"loss:",total_loss)
  print("__________________________________")
  tb.add_hparams(
    {"lr": lr, "bsize": batch_size, "shuffle": shuffle},
    {"accuracy": total_correct/len(train_set), "loss": total_loss}
)

tb.close()

run_id: 1
batch_size: 32 lr: 0.01 shuffle: True
epoch: 0 total_corrct: 48210 loss: 1004.3987563923001
batch_size: 32 lr: 0.01 shuffle: True
epoch: 1 total_corrct: 50665 loss: 796.4051642715931
batch_size: 32 lr: 0.01 shuffle: True
epoch: 2 total_corrct: 51323 loss: 745.2328318655491
batch_size: 32 lr: 0.01 shuffle: True
epoch: 3 total_corrct: 51555 loss: 733.9798557087779
batch_size: 32 lr: 0.01 shuffle: True
epoch: 4 total_corrct: 51749 loss: 709.1487831287086
__________________________________
run_id: 2
batch_size: 32 lr: 0.01 shuffle: False
epoch: 0 total_corrct: 47344 loss: 1056.188487701118
batch_size: 32 lr: 0.01 shuffle: False
epoch: 1 total_corrct: 50465 loss: 822.1729287058115
batch_size: 32 lr: 0.01 shuffle: False
epoch: 2 total_corrct: 50970 loss: 777.3572818636894
batch_size: 32 lr: 0.01 shuffle: False
epoch: 3 total_corrct: 51122 loss: 766.6848340928555
batch_size: 32 lr: 0.01 shuffle: False
epoch: 4 total_corrct: 51290 loss: 741.4896515160799
_____________________________

In [ ]:
  %load_ext tensorboard
  %tensorboard --logdir=runs

<IPython.core.display.Javascript object>